In [5]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

file_folder = r'C:\Users\HP\Downloads\stockmarket_analyze'

In [6]:
pip install meteostat geopy openpyxl

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
# Get weather data

# May need to run the following line in its own code cell one time to install pandasql: ! pip install meteostat
# For details on units that fields returned by meteostat, check out: https://dev.meteostat.net/python/daily.html#data-structure or https://dev.meteostat.net/python/api/units.html for imperial units

from meteostat import Point, Daily, units
# May need to run the following line in its own code cell one time to install pandasql: ! pip install geopy
# This will help us get latitude and longitude for a city
from geopy.geocoders import Nominatim

# This would be set up to be dynamic instead of hard coded
# Often using rolling most completed 12 months in analytics
from datetime import date
end = date.today()
from datetime import datetime, timedelta
start = end - timedelta(days=365)
print(f"Start is {start}")
print(f"End is {end}")
import datetime
start = datetime.datetime(start.year, start.month, start.day)
end = datetime.datetime(end.year, end.month, end.day)

# This is hard coded
import datetime
start = datetime.datetime(2022, 5, 1)
end = datetime.datetime(2023, 4, 30)

# Need to get latitude and longitude for city
geolocator = Nominatim(user_agent="MyApp")
location = geolocator.geocode("New York City, NY")
place = Point(location.latitude, location.longitude)

weather = Daily(place, start, end)
weather = weather.convert(units.imperial)
weather = weather.fetch()
weather["Date"] = weather.index
weather = weather[["Date",
                   "tavg",
                   "prcp",
                   "snow",
                   "wspd",
                   #"wpgt",
                   #"tsun"
                  ]]
weather = weather.reset_index(drop=True)
weather = weather.rename(columns={"tavg": "Avg Temperature (Fahrenheit)", 
                                  "prcp": "Precipitation (Inches)", 
                                  "snow": "Snow (Inches)", 
                                  "wspd": "Avg Windspeed (MPH)", 
                                  #"wpgt": "Peak Wind Gust (MPH)", 
                                  #"tsun": "Minutes of Sunshine"
                                 }, errors="raise")
display(weather)

weather.to_excel(file_folder+"/Weather.xlsx", sheet_name='Weather', index=False) 

Start is 2024-10-17
End is 2025-10-17


GeocoderInsufficientPrivileges: Non-successful status code 403

In [8]:
!pip install yfinance


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# Get stock data

import pandas as pd

# May need to run the following line in its own code cell one time to install pandasql: ! pip install yfinance
# For details on package, check out: https://pypi.org/project/yfinance/#:~:text=To%20download%20price%20history%20into%20one%20table%3A%20import,many%20options%20for%20configuring%20fetching%20and%20processing%2C%20e.g.%3A
# ! pip install -U yfinance pandas
import yfinance as yf

# SPY is a proxy for S&P 500
stock = 'SPY'
ticker = yf.Ticker(stock) 

# get historical market data
stocks = ticker.history(start=start, end=end, auto_adjust = True)

stocks["Date"] = pd.to_datetime(stocks.index)
stocks["Date"] = stocks["Date"].dt.date
stocks = stocks.reset_index(drop=True)
stocks = stocks.sort_values(["Date"], ascending=[True])
stocks = stocks.reset_index(drop=True)
stocks['Previous Close'] = stocks['Close'].shift(1)
stocks = stocks[["Date",
                 "Open",
                 "Close",
                 "Previous Close"
                  ]]
stocks = stocks.reset_index(drop=True)
display(stocks)

stocks.to_excel(file_folder+"/Stocks.xlsx", sheet_name='Stocks', index=False) 

,Date,Open,Close,Previous Close
0,2022-05-02,392.151255,394.444763,NaN
1,2022-05-03,394.949188,396.252960,394.444763
2,2022-05-04,396.919089,408.320007,396.252960
3,2022-05-05,404.027935,393.807098,408.320007
4,2022-05-06,391.228153,391.456543,393.807098
...,...,...,...,...
245,2023-04-24,398.717823,399.337219,398.921112
246,2023-04-25,397.353264,392.998230,399.337219
247,2023-04-26,393.617637,391.333649,392.998230
248,2023-04-27,393.888607,399.124329,391.333649


In [10]:
# Import datasets into Python
import pandas as pd

# Import excel files into Pandas dataframes
# both datasets are one row per day (Stocks only includes days market was open) over the last year or so
Weather = pd.read_excel(open(file_folder + "/Weather.xlsx", 'rb'), sheet_name='Weather') 
Stocks = pd.read_excel(open(file_folder + "/Stocks.xlsx", 'rb'), sheet_name='Stocks') 

In [11]:
stocks

,Date,Open,Close,Previous Close
0,2022-05-02,392.151255,394.444763,NaN
1,2022-05-03,394.949188,396.252960,394.444763
2,2022-05-04,396.919089,408.320007,396.252960
3,2022-05-05,404.027935,393.807098,408.320007
4,2022-05-06,391.228153,391.456543,393.807098
...,...,...,...,...
245,2023-04-24,398.717823,399.337219,398.921112
246,2023-04-25,397.353264,392.998230,399.337219
247,2023-04-26,393.617637,391.333649,392.998230
248,2023-04-27,393.888607,399.124329,391.333649


In [12]:
 ! pip install pandasql


[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [13]:
# Start running SQL against datasets

# May need to run the following line in its own code cell one time to install pandasql: ! pip install pandasql
# Pandasql uses a specific type of SQL called SQLLite
# Here is a reference for date functions in SQLLite: https://www.sqlite.org/lang_datefunc.html

import pandasql

# Select statement
# * pulls all columns
# from statement
df = pandasql.sqldf('''
    Select
    *
    from Weather
''', globals())

display(df)

,Date,Avg Temperature (Fahrenheit),Precipitation (Inches),Snow (Inches),Avg Windspeed (MPH)
0,2022-05-01 00:00:00.000000,56.8,0.000,0.0,5.7
1,2022-05-02 00:00:00.000000,54.3,0.220,0.0,7.5
2,2022-05-03 00:00:00.000000,55.9,0.000,0.0,7.6
3,2022-05-04 00:00:00.000000,55.6,0.150,0.0,5.3
4,2022-05-05 00:00:00.000000,63.3,0.016,0.0,4.2
...,...,...,...,...,...
360,2023-04-26 00:00:00.000000,53.6,0.008,0.0,7.0
361,2023-04-27 00:00:00.000000,53.2,0.094,0.0,4.3
362,2023-04-28 00:00:00.000000,51.8,0.236,0.0,12.1
363,2023-04-29 00:00:00.000000,51.6,1.260,0.0,15.1


In [17]:
# If you want to pull just specific columns, you can list each column separate by commas (no comma between last field and from statement) in the select statement (put double quotes around column names, especially if column has spaces)

df = pandasql.sqldf('''
    Select
    Date,
    "Avg Temperature (Fahrenheit)",
    "Precipitation (Inches)"
    from Weather
''', globals())

display(df) 

,Date,Avg Temperature (Fahrenheit),Precipitation (Inches)
0,2022-05-01 00:00:00.000000,56.8,0.000
1,2022-05-02 00:00:00.000000,54.3,0.220
2,2022-05-03 00:00:00.000000,55.9,0.000
3,2022-05-04 00:00:00.000000,55.6,0.150
4,2022-05-05 00:00:00.000000,63.3,0.016
...,...,...,...
360,2023-04-26 00:00:00.000000,53.6,0.008
361,2023-04-27 00:00:00.000000,53.2,0.094
362,2023-04-28 00:00:00.000000,51.8,0.236
363,2023-04-29 00:00:00.000000,51.6,1.260


In [19]:
# You can use as to rename columns

df = pandasql.sqldf('''
    Select
    "Date",
    "Avg Temperature (Fahrenheit)" as "Avg Temperature",
    "Precipitation (Inches)" as "Precipitation"
    from Weather
''', globals())

display(df) 

,Date,Avg Temperature,Precipitation
0,2022-05-01 00:00:00.000000,56.8,0.000
1,2022-05-02 00:00:00.000000,54.3,0.220
2,2022-05-03 00:00:00.000000,55.9,0.000
3,2022-05-04 00:00:00.000000,55.6,0.150
4,2022-05-05 00:00:00.000000,63.3,0.016
...,...,...,...
360,2023-04-26 00:00:00.000000,53.6,0.008
361,2023-04-27 00:00:00.000000,53.2,0.094
362,2023-04-28 00:00:00.000000,51.8,0.236
363,2023-04-29 00:00:00.000000,51.6,1.260


In [36]:
df = pandasql.sqldf('''
    Select
    "Date",
    "Avg Temperature (Fahrenheit)" as "Avg Temperature",
    "Precipitation (Inches)" as "Precipitation"
    from Weather
    order by "Precipitation (Inches)" desc, "Avg Temperature (Fahrenheit)"
''', globals())

display(df) 

,Date,Avg Temperature,Precipitation
0,2022-10-04 00:00:00.000000,52.3,1.772
1,2023-04-30 00:00:00.000000,55.8,1.626
2,2022-09-25 00:00:00.000000,64.6,1.441
3,2022-12-23 00:00:00.000000,36.3,1.260
4,2023-04-29 00:00:00.000000,51.6,1.260
...,...,...,...
360,2022-07-13 00:00:00.000000,81.7,0.000
361,2022-05-31 00:00:00.000000,82.4,0.000
362,2022-07-19 00:00:00.000000,83.3,0.000
363,2022-07-22 00:00:00.000000,85.6,0.000


In [20]:
# A where statement can be used to filter rows


df = pandasql.sqldf('''
  Select
  "Datae",
  "Avg Temperature (Fahrenheit)" as "Avg Temperature",
  "Precipitation (Inches)" as "Precipitation"
  from Weather
  where "Avg Temperature (Fahrenheit)" < 20 
  order by "Avg Temperature (Fahrenheit)" asc
''', globals())

display(df) 
  

,"""Datae""",Avg Temperature,Precipitation
0,Datae,13.8,0.0
1,Datae,17.6,0.0


In [22]:
# A where statements can have multiple and and or statements, but be sure to use parentheses around or statements appropriately

# Cold days where there was also either a good amount of rain or a good amount of wind
df = pandasql.sqldf('''
    Select
    "Date",
    "Avg Temperature (Fahrenheit)" as "Avg Temperature",
    "Precipitation (Inches)" as "Precipitation",
    "Avg Windspeed (MPH)" as "Avg Windspeed"
    from Weather
    where "Avg Temperature (Fahrenheit)"<50 and "Precipitation (Inches)">1 and "Avg Windspeed (MPH)">10
''', globals())

display(df) 

,Date,Avg Temperature,Precipitation,Avg Windspeed
0,2022-05-07 00:00:00.000000,49.5,1.209,16.5
1,2022-12-23 00:00:00.000000,36.3,1.260,18.3


In [23]:
# The in() (and non in()) statements
# When filtering on date in SQLlite3 (not needed in other versions of SQL), have to put date field and date value in date()

df = pandasql.sqldf('''
    Select
    "Date",
    "Avg Temperature (Fahrenheit)" as "Avg Temperature",
    "Precipitation (Inches)" as "Precipitation",
    "Snow (Inches)" as "Snow",
    "Avg Windspeed (MPH)" as "Avg Windspeed"
    from Weather
     where date("Date")  not in(date('2022-12-24'),date('2022-12-25'),date('2022-12-26'))
    /*where date("Date") in(date('2022-12-24'),date('2022-12-25'),date('2022-12-26'))*/
    /*  where date("Date")=date('2022-12-24') or date("Date")=date('2022-12-25') or date("Date")=date('2022-12-26')*/
''', globals())

display(df) 

,Date,Avg Temperature,Precipitation,Snow,Avg Windspeed
0,2022-05-01 00:00:00.000000,56.8,0.000,0.0,5.7
1,2022-05-02 00:00:00.000000,54.3,0.220,0.0,7.5
2,2022-05-03 00:00:00.000000,55.9,0.000,0.0,7.6
3,2022-05-04 00:00:00.000000,55.6,0.150,0.0,5.3
4,2022-05-05 00:00:00.000000,63.3,0.016,0.0,4.2
...,...,...,...,...,...
357,2023-04-26 00:00:00.000000,53.6,0.008,0.0,7.0
358,2023-04-27 00:00:00.000000,53.2,0.094,0.0,4.3
359,2023-04-28 00:00:00.000000,51.8,0.236,0.0,12.1
360,2023-04-29 00:00:00.000000,51.6,1.260,0.0,15.1


In [24]:
# The between statement can also be used for date and numeric fields

df = pandasql.sqldf('''
    Select
    "Date",
    "Avg Temperature (Fahrenheit)" as "Avg Temperature",
    "Precipitation (Inches)" as "Precipitation",
    "Snow (Inches)" as "Snow",
    "Avg Windspeed (MPH)" as "Avg Windspeed"
    from Weather
    where date("Date") between date('2022-12-24') and date('2022-12-26')
    
''', globals())

display(df) 

,Date,Avg Temperature,Precipitation,Snow,Avg Windspeed
0,2022-12-24 00:00:00.000000,13.8,0.0,0.0,19.8
1,2022-12-25 00:00:00.000000,21.7,0.0,0.0,15.8
2,2022-12-26 00:00:00.000000,24.4,0.0,0.0,9.8


In [25]:
# To filter on whether a field is null (missing) or not, use is null or is not null


df = pandasql.sqldf('''
  Select
  "Date",
  "Avg Temperature (Fahrenheit)" as "Avg Temperature",
  "Precipitation (Inches)" as "Precipitation",
  "Snow (Inches)" as "Snow",
  "Avg Windspeed (MPH)" as "Avg Windspeed"
   from Weather
   where "Snow (Inches)" is null
   /*where  "Snow (Inches)" is not null*/
   /*where  not("Snow (Inches)" is  null*/
''',globals())

display(df) 
  
    

,Date,Avg Temperature,Precipitation,Snow,Avg Windspeed


In [27]:
# functions can be applied to columns in table in select statement to create new columns
# Extracting dates usually looks a little different in other version of SQL compared to SQLlite syntax below
# Adding null values results in null values, but can use coalesce to replace null values
# Notice the use of nesting functions in the last column created (Rain and Snow Imputted) below

df = pandasql.sqldf('''
    Select
    "Date",
    strftime('%Y', "Date") AS "Year",
    "Avg Temperature (Fahrenheit)" as "Avg Temperature",
    "Avg Temperature (Fahrenheit)"/2 as "Avg Temperature Divided by 2",
    "Precipitation (Inches)" as "Precipitation",
    "Snow (Inches)" as "Snow",
    "Precipitation (Inches)" + "Snow (Inches)" as "Rain and Snow",
    coalesce("Snow (Inches)",0) as "Snow Imputted",
    "Precipitation (Inches)" + coalesce("Snow (Inches)",0) as "Rain and Snow Imputted"
    from Weather
''', globals())

display(df) 
  
   
  

,Date,Year,Avg Temperature,Avg Temperature Divided by 2,Precipitation,Snow,Rain and Snow,Snow Imputted,Rain and Snow Imputted
0,2022-05-01 00:00:00.000000,2022,56.8,28.40,0.000,0.0,0.000,0.0,0.000
1,2022-05-02 00:00:00.000000,2022,54.3,27.15,0.220,0.0,0.220,0.0,0.220
2,2022-05-03 00:00:00.000000,2022,55.9,27.95,0.000,0.0,0.000,0.0,0.000
3,2022-05-04 00:00:00.000000,2022,55.6,27.80,0.150,0.0,0.150,0.0,0.150
4,2022-05-05 00:00:00.000000,2022,63.3,31.65,0.016,0.0,0.016,0.0,0.016
...,...,...,...,...,...,...,...,...,...
360,2023-04-26 00:00:00.000000,2023,53.6,26.80,0.008,0.0,0.008,0.0,0.008
361,2023-04-27 00:00:00.000000,2023,53.2,26.60,0.094,0.0,0.094,0.0,0.094
362,2023-04-28 00:00:00.000000,2023,51.8,25.90,0.236,0.0,0.236,0.0,0.236
363,2023-04-29 00:00:00.000000,2023,51.6,25.80,1.260,0.0,1.260,0.0,1.260


In [28]:
# Notice SQL mult-line comment below that opens with /* and closes with */


df = pandasql.sqldf('''
    Select
    "Date",
    "Avg Temperature (Fahrenheit)" as "Avg Temperature",
    "Precipitation (Inches)" as "Precipitation",
    "Snow (Inches)" as "Snow",
    "Avg Windspeed (MPH)" as "Avg Windspeed",
    case when "Avg Temperature (Fahrenheit)"<40 then 1
         when "Precipitation (Inches)">1 then 1
         when "Avg Windspeed (MPH)">20 then 1
         else 0 
    end as "Bad Weather"
    /*
    could also be written as...
    case when "Avg Temperature (Fahrenheit)"<40 or "Precipitation (Inches)">1 or "Avg Windspeed (MPH)">20 then 1 else 0 end as "Bad Weather"
    */
    from Weather
''', globals())

display(df) 

,Date,Avg Temperature,Precipitation,Snow,Avg Windspeed,Bad Weather
0,2022-05-01 00:00:00.000000,56.8,0.000,0.0,5.7,0
1,2022-05-02 00:00:00.000000,54.3,0.220,0.0,7.5,0
2,2022-05-03 00:00:00.000000,55.9,0.000,0.0,7.6,0
3,2022-05-04 00:00:00.000000,55.6,0.150,0.0,5.3,0
4,2022-05-05 00:00:00.000000,63.3,0.016,0.0,4.2,0
...,...,...,...,...,...,...
360,2023-04-26 00:00:00.000000,53.6,0.008,0.0,7.0,0
361,2023-04-27 00:00:00.000000,53.2,0.094,0.0,4.3,0
362,2023-04-28 00:00:00.000000,51.8,0.236,0.0,12.1,0
363,2023-04-29 00:00:00.000000,51.6,1.260,0.0,15.1,1


In [29]:
# Adding distinct after select statement will only pull back distinct combinations of values in selected columns
df = pandasql.sqldf('''
    Select
    distinct
    "Snow (Inches)" as "Snow"
    from Weather
''', globals())

display(df) 

,Snow
0,0.000
1,0.197
2,0.118


In [30]:
# Aggregation in SQL allows you to summarize columns across 0's using count (non-null values), sum, max, min, avg
df = pandasql.sqldf('''
    Select
    count(*) as "Total Rows", /* Count * counts all the rows in a table */
    count("Date") as "Count of Date",
    count("Snow (Inches)") as "Count of Snow",
    min("Date") as "Min Date",
    max("Date") as "Max Date",
    min("Avg Temperature (Fahrenheit)") as "Min Avg Temperature",
    avg("Avg Temperature (Fahrenheit)") as "Avg Avg Temperature",
    max("Avg Temperature (Fahrenheit)") as "Max Avg Temperature",
    sum("Precipitation (Inches)") as "Total Precipitation"
    from Weather
''', globals())

display(df) 

,Total Rows,Count of Date,Count of Snow,Min Date,Max Date,Min Avg Temperature,Avg Avg Temperature,Max Avg Temperature,Total Precipitation
0,365,365,365,2022-05-01 00:00:00.000000,2023-04-30 00:00:00.000000,13.8,57.057534,87.4,46.349


In [31]:
# Count distinct can tell you the number of unique values in a table; this is a good way to check for duplicates on a column that you think is a unique identifier for each row
# Notice count will not count the null Snow value

df = pandasql.sqldf('''
    Select
    count(*) as "Total Rows",
    count(distinct "Date") as "Distinct Dates",
    count(distinct "Snow (Inches)") as "Distinct Snow Values"
    from Weather
''', globals())

display(df) 

,Total Rows,Distinct Dates,Distinct Snow Values
0,365,365,3


In [32]:
# You may want to aggregate columns within values of another variable; we can use group by for this

df = pandasql.sqldf('''
    Select
    strftime('%Y', "Date")*100+strftime('%m', "Date") AS "Year Month",
    count(*) as "Dates in Month",
    avg("Avg Temperature (Fahrenheit)") as "Avg Temperature",
    avg("Precipitation (Inches)") as "Avg Daily Precipitation",
    avg(coalesce("Snow (Inches)",0)) as "Avg Daily Snow",
    avg("Avg Windspeed (MPH)") as "Avg Windspeed"
    from Weather
    group by strftime('%Y', "Date")*100+strftime('%m', "Date")
    order by strftime('%Y', "Date")*100+strftime('%m', "Date")
''', globals())

display(df) 

,Year Month,Dates in Month,Avg Temperature,Avg Daily Precipitation,Avg Daily Snow,Avg Windspeed
0,202205,31,63.658065,0.167161,0.000000,6.819355
1,202206,30,71.220000,0.118333,0.000000,7.390000
2,202207,31,79.825806,0.074548,0.000000,6.254839
3,202208,31,77.700000,0.072516,0.000000,6.416129
4,202209,30,68.120000,0.138500,0.000000,6.826667
5,202210,31,55.829032,0.192484,0.000000,7.677419
6,202211,30,48.746667,0.089700,0.000000,7.586667
7,202212,31,36.367742,0.149968,0.000000,8.438710
8,202301,31,40.758065,0.145355,0.000000,6.503226
9,202302,28,41.360714,0.054214,0.007036,6.817857


In [33]:
display(Weather.head())

df = pandasql.sqldf('''
    Select
    count(*) as "Total Rows",
    count(distinct "Date") as "Distinct Dates",
    avg(case when "Avg Temperature (Fahrenheit)"<40 or "Precipitation (Inches)">1 or "Avg Windspeed (MPH)">20 then 1 else 0 end) as "% Days Bad Weather"
    from Weather
''', globals())

display(df)

display(Stocks.head())

df = pandasql.sqldf('''
    Select
    count(*) as "Total Rows",
    count(distinct "Date") as "Distinct Dates",
    avg(("Close"/"Previous Close")-1) as "Avg Return"
    from Stocks
''', globals())

display(df) 

,Date,Avg Temperature (Fahrenheit),Precipitation (Inches),Snow (Inches),Avg Windspeed (MPH)
0,2022-05-01,56.8,0.000,0.0,5.7
1,2022-05-02,54.3,0.220,0.0,7.5
2,2022-05-03,55.9,0.000,0.0,7.6
3,2022-05-04,55.6,0.150,0.0,5.3
4,2022-05-05,63.3,0.016,0.0,4.2


,Total Rows,Distinct Dates,% Days Bad Weather
0,365,365,0.189041


,Date,Open,Close,Previous Close
0,2022-05-02,392.151255,394.444763,NaN
1,2022-05-03,394.949188,396.252960,394.444763
2,2022-05-04,396.919089,408.320007,396.252960
3,2022-05-05,404.027935,393.807098,408.320007
4,2022-05-06,391.228153,391.456543,393.807098


,Total Rows,Distinct Dates,Avg Return
0,250,250,0.000182


In [34]:
# Joining tables - types of joins
# aliases / on statement

df = pandasql.sqldf('''
    Select
    s."Date",
    w."Avg Temperature (Fahrenheit)",
    w."Precipitation (Inches)",
    w."Avg Windspeed (MPH)",
    case when w."Avg Temperature (Fahrenheit)"<40 or w."Precipitation (Inches)">1 or w."Avg Windspeed (MPH)">20 then 1 else 0 end as "Bad Weather Day",
    s."Close",
    (s."Close"/s."Previous Close")-1 as "Return"    
    from Stocks s
    inner join Weather w on s."Date" = w."Date"
    order by s."Date"
''', globals())

display(df) 

,Date,Avg Temperature (Fahrenheit),Precipitation (Inches),Avg Windspeed (MPH),Bad Weather Day,Close,Return
0,2022-05-02 00:00:00.000000,54.3,0.220,7.5,0,394.444763,NaN
1,2022-05-03 00:00:00.000000,55.9,0.000,7.6,0,396.252960,0.004584
2,2022-05-04 00:00:00.000000,55.6,0.150,5.3,0,408.320007,0.030453
3,2022-05-05 00:00:00.000000,63.3,0.016,4.2,0,393.807098,-0.035543
4,2022-05-06 00:00:00.000000,55.8,0.941,5.3,0,391.456543,-0.005969
...,...,...,...,...,...,...,...
245,2023-04-24 00:00:00.000000,53.8,0.000,4.2,0,399.337219,0.001043
246,2023-04-25 00:00:00.000000,53.8,0.000,3.9,0,392.998230,-0.015874
247,2023-04-26 00:00:00.000000,53.6,0.008,7.0,0,391.333649,-0.004236
248,2023-04-27 00:00:00.000000,53.2,0.094,4.3,0,399.124329,0.019908


In [35]:
df = pandasql.sqldf('''
    Select
    case when w."Avg Temperature (Fahrenheit)"<40 or w."Precipitation (Inches)">1 or w."Avg Windspeed (MPH)">20 then 1 else 0 end as "Bad Weather Day",
    count(*) as "Days",
    avg((s."Close"/s."Previous Close")-1) as "Avg Return"    
    from Stocks s
    inner join Weather w on s."Date" = w."Date"
    group by case when w."Avg Temperature (Fahrenheit)"<40 or w."Precipitation (Inches)">1 or w."Avg Windspeed (MPH)">20 then 1 else 0 end
    order by case when w."Avg Temperature (Fahrenheit)"<40 or w."Precipitation (Inches)">1 or w."Avg Windspeed (MPH)">20 then 1 else 0 end
''', globals())

display(df) 

,Bad Weather Day,Days,Avg Return
0,0,209,-0.000174
1,1,41,0.001991
